In [1]:
import mtcnn
import cv2
from mtcnn.mtcnn import MTCNN
detector=MTCNN()
import matplotlib.pyplot as plt
import os
from datetime import datetime


Using TensorFlow backend.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K
import os

In [3]:
datetime.now()

datetime.datetime(2020, 1, 17, 16, 31, 6, 196897)

In [4]:

#Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [5]:
model.load_weights('vgg_face_weights.h5')

In [6]:
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

In [7]:
classifier_model=tf.keras.models.load_model('home_face_classifier_model.h5')

In [8]:
test_images_path='D:/data/val/'

In [9]:
person_rep={0: 'ayush', 1: 'unique', 2: 'utkarsh'}

In [13]:
def plot(img):
    plt.figure(figsize=(8,4))
    plt.imshow(img[:,:,::-1])
    plt.show()

In [14]:
def face_extractor(img):
    faces=face_cascade.detectMultiScale(img,1.3,5)
    if faces in ():
        return None
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cropped_face=img[y:y+h,x:x+w]

In [18]:
video_capture=cv2.VideoCapture(0)

In [19]:
time_record_ayush=[]
time_record_unique=[]
time_record_utkarsh=[]

In [20]:
while True:
    _,img=video_capture.read()

    result=detector.detect_faces(img)
# in the result we will have multiple number of dictionaries for each image, and we need to extract the bounding box of the face.      
    for x in result:
        bounding_box=x['box']
#after extracting bounding box, draw rectangle around the face.
        cv2.rectangle(img,(bounding_box),(0,0,255),3)
#save the image.
        x1,y3,w,h=x['box']
        x2,y2=x1+w,y3+h
        cropped=img[y3:y2, x1:x2]
        if type(cropped) is np.ndarray:
            img_c=cv2.resize(cropped,(224,224))
        cv2.imwrite('D:/data/temp/crop_img.jpg',img_c)
            # Get Embeddings
        crop_img=load_img('D:/data/temp/crop_img.jpg',target_size=(224,224))
        crop_img=img_to_array(crop_img)
        crop_img=np.expand_dims(crop_img,axis=0)
        crop_img=preprocess_input(crop_img)
        img_encode=vgg_face(crop_img)
        embed=K.eval(img_encode)
        person=classifier_model.predict(embed)
        if person[0][0]>.80:
            name=person_rep[0]
        elif person[0][1]>.80:
            name=person_rep[1]
        elif person[0][2]>.90:
            name=person_rep[2]
        else:
            name='unknown'
        os.remove('D:/data/temp/crop_img.jpg')
        cv2.rectangle(img,(bounding_box),(0,0,255),3)
        img=cv2.putText(img,name,(x1,y3-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
        img=cv2.putText(img,str(np.max(person)),(x2,y2+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
        if name !='unknown':
            if name== 'ayush':
                time_record_ayush.append(str(datetime.now()))
            elif name=='unique':
                time_record_unique.append(str(datetime.now()))
            elif name=='utkarsh':
                time_record_utkarsh.append(str(datetime.now()))
            #time_record={name:}
        else:
            pass
  # Save images with bounding box,name and accuracy 
    cv2.imshow('video',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()
time_stamps={'ayush':time_record_ayush,'unique':time_record_unique,'utkarsh':time_record_utkarsh}


In [21]:

time_stamps


{'ayush': [],
 'unique': ['2020-01-17 16:31:45.884005'],
 'utkarsh': ['2020-01-17 16:31:31.174355',
  '2020-01-17 16:31:36.736503',
  '2020-01-17 16:31:37.304603',
  '2020-01-17 16:31:37.889483',
  '2020-01-17 16:31:38.609939',
  '2020-01-17 16:31:44.661006',
  '2020-01-17 16:31:48.846465',
  '2020-01-17 16:31:49.531955',
  '2020-01-17 16:31:50.754927',
  '2020-01-17 16:31:51.353478',
  '2020-01-17 16:31:53.110488',
  '2020-01-17 16:31:53.694404']}

In [28]:
video_capture.release()
cv2.destroyAllWindows()